In [39]:
import sys
from pathlib import Path
project_path = str(Path.cwd().parent)
sys.path.append(project_path)


import pymysql
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

import sys
import requests
from bs4 import BeautifulSoup
from tabs import tab1, tab2, tab3, tab4, tab5
from agents import get_user_agents

from schemas.general import GeneralCreateSchema
from schemas.subscriber import SubscriberCreateSchema
from schemas.financial import FinancialCreateSchema
from schemas.shareholder import ShareholderCreateSchema
from schemas.prediction import PredictionCreateSchema

from db import models
from sqlalchemy import and_ 
from settings import DB_URL, DELISTING
from utilities.session import session_scope
import re
from datetime import datetime

import requests
from bs4 import BeautifulSoup

from agents import get_user_agents
from utilities.time_measure import timeit


# @timeit
def scrape_company_codes(year=2021):

    current_year = datetime.now().year
    codes = []
    for year in range(year, current_year + 1):
        # '리스트가 없습니다.'라는 colspan=9 속성이 안나올때까지 순회하게 됨.
        page_data = None
        page = 1
        # while page_data is None:
        url = "http://www.ipostock.co.kr/sub03/ipo02.asp?str4=2022&str5=all"
        # url = f"http://www.ipostock.co.kr/sub03/ipo02.asp?str4={year}&str5=all&page={page}"
        # page += 1

        from utilities import request_helper

        req = request_helper.requests_retry_session().get(url, timeout=5)
        soup = BeautifulSoup(req.content, "lxml", )

        page_data = soup.select_one('td[colspan="9"]')
        # if page_data is not None:
        #     break

        a_tags = soup.select('a[href*="view_pg"][class=""]')

        for tag in a_tags:
            href = tag["href"]
            code = re.search("code=(.+)&", href).group(1)
            codes.append(code)
    return codes



In [17]:
url = "http://www.ipostock.co.kr/sub03/ipo02.asp?str4=2022&str5=all"
# url = f"http://www.ipostock.co.kr/sub03/ipo02.asp?str4={year}&str5=all&page={page}"
# page += 1

from utilities import request_helper

req = request_helper.requests_retry_session().get(url, timeout=5)
soup = BeautifulSoup(req.content, "lxml", )

In [18]:
import bs4

table = soup.select_one('#print > table >  tr:nth-child(4) > td > table >  tr:nth-child(3) > td > table >  tr:nth-child(4) > td > table')
for idx, tr in enumerate(table, 1):
    if isinstance(tr, bs4.element.Tag):
        td = tr.find('td', {'width': '120'})
        if td is not None:
            delisting = td.text.strip()
            if delisting == '공모철회':
                td_name = tr.find('td', {'width': '*'})
                if td_name is not None:
                    company_name = td_name.select_one('table > tr > td:nth-child(2) > a > font').text.strip().replace('.', '')
                    print(company_name)

유안타제12호스팩
자람테크놀로지
유안타제11호스팩
바이오인프라
미래에셋드림스팩1호
밀리의서재


In [40]:

from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from db import models
from utilities.session import session_scope


ci_name = '래몽래인'
DELISTING = '공모철회'

engine = create_engine(DB_URL, pool_recycle=3600)
Session = sessionmaker(bind=engine)

with session_scope(Session) as session:
    result = session.query(models.CompanyInfoGeneral).all()
    for row in result:
        print(row)



In [117]:
DELISTING = 'a'
with session_scope(Session) as session:
    result = session.query(models.CompanyInfoGeneral)\
        .filter(and_(models.CompanyInfoGeneral.ci_name == ci_name, models.CompanyInfoGeneral.ci_demand_forecast_date != DELISTING))\
        .update({models.CompanyInfoGeneral.ci_demand_forecast_date: DELISTING})
    print(type(result), resul   t)
    
    session.commit()
    print(type(result), result)

<class 'int'> 2
<class 'int'> 2


In [37]:
ci_name = '래몽래인'
with session_scope(Session) as session:
    print(ci_name)
    result = session.query(models.CompanyInfoGeneral)\
        .filter(
            and_(
                models.CompanyInfoGeneral.ci_name == ci_name,
                # models.CompanyInfoGeneral.ci_demand_forecast_date != DELISTING,
            )
        )\
        .first()
    print(result)


래몽래인
None


In [35]:
import pymysql
from settings import DB_URL
# Connect to the database
_url = DB_URL
url = _url.replace('test_trustinv', 'trustinv').replace('mariadb+', '')

conn = pymysql.connect(
    host='ipolistingdb.civxowlytnik.ap-northeast-2.rds.amazonaws.com',
    port=3306,
    user='admin',
    password='dara100400!',
    db='trustinv'
)

# Create a cursor
cur = conn.cursor()

# Execute the query
cur.execute("SELECT * FROM company_info_general")

# Fetch all the results
result = cur.fetchall()

# Iterate through the results and print them
for row in result:
    print(row)

# Close the cursor
cur.close()

# Close the connection
conn.close()

(210, '코스닥', '', '디어유', '376300', 'http://43.200.190.19/data/company/210_20211116121720.jpg', '210_20211116121720.jpg', '코스닥', '테슬라요건 상장 ', '팬슈머', '메타버스', '신규상장', '2021/06/11', '2021/09/15', 500, '안종오', '02-6925-2385', 'www.dear-u.co', '2017/07/04', '일반', '111-87-00747', 12, 55, '소프트웨어 개발 및 공급업', '모바일 어플리케이션(Dear U bubbl..', '통일주권', '한국투자증권', '서울특별시 강남구 삼성로 633, 4층(삼성동, 삼광빌딩)', 13048.0, -1352.0, -2250.0, -2268.0, '(주)에스엠스튜디오스', 40.0, '', 3300000, None, 16812769, 84.0, 16812769, 100.0, 20168324, '', "모바일 플랫폼 '버블서비스' 팬덤 비즈니스 운영중이며 향후 메타버스 시장 성장에 따라 지속 성장이 기대됩니다.", '', '2021/10/25~2021/10/26', '2021.11.01~2021.11.02', '2021/11/04', '2021/11/04', '2021/11/10', 27332, '18,000~24,000', '594~792', 26000, 858.0, '개인 50% / 수요예측 참여기관 0%', '1598:1', None, None, None, '3,300,000 주', 1980000, 60, 495000, 15, 825000, 25, 0, 0, '', 'Y', '', 'Y', '', '', '', '', '', 0, '', '2000:1', 26, '참여건수기준\r\n\r\n[15일: 37건]\r\n[1개월: 195건]\r\n[3개월: 244건]\r\n[6개월: 161건]', 36.13, 5, 3, 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 